In [ ]:
import pandas

#convert the sites into a df
sitesdf = pandas.DataFrame(columns=('icon', 'title', 'url', 'description'))
f = open("hkssites.txt", "r")
row = 0
icon =""
title=""
url=""
desc=""
for x in f:
    x =x.strip()
    if x == "":
        continue
    if len(x) == 1 or len(x) == 2:
        #add previous site and reset
        if  title+url != "":
            sitesdf.loc[row] = [icon,title,url,desc]
            row+=1
            title=""
            url=""
        icon=x    
    if "http" in x:
        url = x
    if url == "":
        title =x
    else :
        desc = x
    
f.close()
sitesdf = sitesdf.reset_index()  # make sure indexes pair with number of rows
print(sitesdf.shape[0])
print(sitesdf.tail(1))

In [6]:
import pandas as pd
#read from csv, clean and filter
sitesdf= pd.read_csv("knowledgesites.csv",keep_default_na=False)
# f = open("knowledgesites.txt", "r")
# urls = f.readlines()
print (sitesdf)

                                                  url
0          https://hksinc.sharepoint.com/sites/Energy
1   https://hksinc.sharepoint.com/sites/EnergyMode...
2   https://hksinc.sharepoint.com/sites/EquitableC...
3   https://hksinc.sharepoint.com/sites/EquityinEd...
4     https://hksinc.sharepoint.com/sites/Flexibility
5          https://hksinc.sharepoint.com/sites/Gender
6             https://hksinc.sharepoint.com/sites/GIS
7    https://hksinc.sharepoint.com/sites/HealthandArt
8   https://hksinc.sharepoint.com/sites/Healthcare...
9         https://hksinc.sharepoint.com/sites/Healthy
10  https://hksinc.sharepoint.com/sites/HealthyEnv...
11            https://hksinc.sharepoint.com/sites/hpd
12  https://hksinc.sharepoint.com/sites/InclusiveD...
13           https://hksinc.sharepoint.com/sites/JEDI
14  https://hksinc.sharepoint.com/sites/LeadingWit...
15     https://hksinc.sharepoint.com/sites/MassTimber
16     https://hksinc.sharepoint.com/sites/Materials/
17            https://hksinc

In [2]:
#set up credentials + imports
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
import os
from bs4 import BeautifulSoup
from dotenv import load_dotenv

#set up credentials
load_dotenv(".env", override=True)
userCredential = UserCredential(os.getenv('user'),os.getenv('password') )


In [3]:
def sitepages(ctx):
    #list pages relative to site_url
    site_pages = ctx.site_pages.pages.get().execute_query()
    # for site_page in site_pages:  # type: SitePage
    #     print(site_page.file_name)
    print(len(site_pages))
    return site_pages

In [4]:
# returns the content from url
def content(ctx, relative_url):
    file = ctx.web.get_file_by_server_relative_url(relative_url)
    file_item = file.listItemAllFields.select(["CanvasContent1", "LayoutWebpartsContent"]).get().execute_query()
    
    #print(content)
    return file_item.properties.get("CanvasContent1")

In [7]:

# iterates all the sites from the txt file gets site pages and then content from each and stores to a text file
# Create a directory to store the text files
if not os.path.exists("text/"):
        os.mkdir("text/")

if not os.path.exists("text/hksinc.sharepoint.com/"):
        os.mkdir("text/hksinc.sharepoint.com/")

# Create a directory to store the csv files
if not os.path.exists("processed"):
        os.mkdir("processed")
        
#iterate sites        
for index, row in sitesdf.iterrows():
    site_url = row["url"]
    ctx = ClientContext(site_url).with_credentials(userCredential)
    web = ctx.web
    ctx.load(web)
    try :
        ctx.execute_query()
    except :
         continue
    print(site_url)
    #print(site_url.replace("https://hksinc.sharepoint.com",""))
    for page in sitepages(ctx):
        relative_url = site_url.replace("https://hksinc.sharepoint.com","")+ "/SitePages/"+page.file_name
        
        #get content for site page
        cleanUrl = relative_url.replace("/", "_").replace("?", "_").replace(":", "_").replace(".", "_")
        #print(cleanUrl)
        try:
            t = content(ctx, relative_url)
        except:
            print("no content")
        else:
            if t is not None:
                soup = BeautifulSoup(t)
                    # Get the text but remove the tags
                text = soup.get_text()
                if text !="":
                    with open('text/hksinc.sharepoint.com/'+cleanUrl + ".txt", "w", encoding="utf-8") as f:    
                        

                        # If the crawler gets to a page that requires JavaScript, it will stop the crawl
                        if ("You need to enable JavaScript to run this app." in text):
                            print("Unable to parse page " + url + " due to JavaScript being required")
                        
                        # Otherwise, write the text to the file in the text directory
                        f.write(text)

https://hksinc.sharepoint.com/sites/EnergyModeling
7
https://hksinc.sharepoint.com/sites/EquitableCommunities
11
https://hksinc.sharepoint.com/sites/EquityinEducation
1
https://hksinc.sharepoint.com/sites/Flexibility
3
https://hksinc.sharepoint.com/sites/GIS
1
https://hksinc.sharepoint.com/sites/HealthandArt
1
https://hksinc.sharepoint.com/sites/HealthcareWorkers
1


In [16]:
#checking the process on the base url and relative url
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext
site_url = "https://hksinc.sharepoint.com"
ctx = ClientContext(site_url).with_credentials(userCredential)
web = ctx.web
ctx.load(web)
ctx.execute_query()
print("Web title: {0}".format(web.properties['Title'])) 

file = ctx.web.get_file_by_server_relative_url("/SitePages/Breaking-News--Fast-Company-Announces-Project-Connect.aspx")

file_item = file.listItemAllFields.select(["CanvasContent1", "LayoutWebpartsContent","BannerImageUrl"]).get().execute_query()
print(file_item.properties.get("BannerImageUrl"))

Web title: Source
{'Description': 'https://hksinc.sharepoint.com/_layouts/15/getpreview.ashx?guidSite=1d26113b-6e40-4404-814d-038f1605cea0&guidWeb=501cc4e4-f8f8-43a7-9b83-45e8756fa5db&guidFile=1cb893cc-4766-41bc-80a4-22aea687cd9f', 'Url': 'https://hksinc.sharepoint.com/_layouts/15/getpreview.ashx?guidSite=1d26113b-6e40-4404-814d-038f1605cea0&guidWeb=501cc4e4-f8f8-43a7-9b83-45e8756fa5db&guidFile=1cb893cc-4766-41bc-80a4-22aea687cd9f'}


In [19]:
#list pages relative to site_url
site_pages = ctx.site_pages.pages.get().execute_query()

print(len(site_pages))
for site_page in site_pages:  # type: SitePage
    print(site_page.file_name)
    
    

11
Home.aspx
Overview.aspx
Process.aspx
Synthesis-and-Documentation.aspx
Data.aspx
Fulcrum.aspx
Annotated-Plan.aspx
Photo-Essay.aspx
Shadowing.aspx
Behavior-Mapping.aspx
Interviewing.aspx
